In [4]:
import random
def xor(l1,l2):
    res=0
    for i in range(len(l1)):
        res+=l1[i]*l2[i]
    if res%2==1:
        return 1
    else:
        return 0
def distance(l1,l2):
    res=0
    for i in range(len(l1)):
        if l1[i]!=l2[i]:
            res+=1
    return res

            
def create_table(impulse):
    table=[[0 for c in range(5+len(impulse))] for k in range(8)]
    num=2**3
    k=5+len(impulse)
    for i in range(num):
        binary=format(i,"b")
        
        table[i][2]=int(binary[-1])
        if len(binary)>1:
            table[i][1]=int(binary[-2])
        if len(binary)>2:
            table[i][0]=int(binary[-3])
            
            
        if table[i][1]==0 and table[i][2]==0:#state
            table[i][3]="a"
        elif  table[i][1]==0 and table[i][2]==1:
            table[i][3]="b"
        elif  table[i][1]==1 and table[i][2]==0:
            table[i][3]="c"
        else:
            table[i][3]="d"
            
        if table[i][0]==0 and table[i][1]==0:#after state
            table[i][k-1]="a"
        elif  table[i][0]==0 and table[i][1]==1:
            table[i][k-1]="b"
        elif  table[i][0]==1 and table[i][1]==0:
            table[i][k-1]="c"
        else:
            table[i][k-1]="d"
        for j in range(len(impulse)):
            table[i][4+j]=xor(table[i][0:3],impulse[j])
        
       # table[i][4]=xor(table[i][0:3],g[0])
       # table[i][5]=xor(table[i][0:3],g[1])
       # table[i][6]=xor(table[i][0:3],g[2])
            
    return table

def decode(bit,impulse):
    table=create_table(impulse)
    n=len(impulse)
    num=len(bit)//n
    #print(num)
    v=[[0 for i in range(num+1)] for i in range(4)]
    s=[["0" for i in range(num+1)] for i in range(4)]
        
    for i in range(num):
        bit_seg=bit[n*i:n*(i+1)]
        if i==0:
            v[0][i+1]=distance(bit_seg,table[0][4:4+n])
            s[0][i+1]="a"
            v[2][i+1]=distance(bit_seg,table[4][4:4+n])
            s[2][i+1]="a"
        elif i==1:
            #print(v[0][i])
            v[0][i+1]=v[0][i]+distance(bit_seg,table[0][4:4+n])#a to a in 0
            s[0][i+1]="a"
            
            v[1][i+1]=v[2][i]+distance(bit_seg,table[2][4:4+n])#c to b in 0
            s[1][i+1]="c"
            
            v[2][i+1]=v[0][i]+distance(bit_seg,table[4][4:4+n])#a to c in 1
            s[2][i+1]="a"
            
            v[3][i+1]=v[2][i]+distance(bit_seg,table[6][4:4+n])#c to d in 1
            s[3][i+1]="c"
        else:
            # a or b to a in 0
            v[0][i+1]=min(v[0][i]+distance(bit_seg,table[0][4:4+n]),v[1][i]+distance(bit_seg,table[1][4:4+n]))
            if v[0][i+1]==v[0][i]+distance(bit_seg,table[0][4:4+n]):
                s[0][i+1]="a"
            else:
                s[0][i+1]="b"
                
            # c or d to b in 0
            v[1][i+1]=min(v[2][i]+distance(bit_seg,table[2][4:4+n]),v[3][i]+distance(bit_seg,table[3][4:4+n]))
            if v[1][i+1]==v[2][i]+distance(bit_seg,table[2][4:4+n]):
                s[1][i+1]="c"
            else:
                s[1][i+1]="d"
                
            # a or b to c in 1
            v[2][i+1]=min(v[0][i]+distance(bit_seg,table[4][4:4+n]),v[1][i]+distance(bit_seg,table[5][4:4+n]))
            if v[2][i+1]==v[0][i]+distance(bit_seg,table[4][4:4+n]):
                s[2][i+1]="a"
            else:
                s[2][i+1]="b"
                
            # c or d to d in 1
            v[3][i+1]=min(v[2][i]+distance(bit_seg,table[6][4:4+n]),v[3][i]+distance(bit_seg,table[7][4:4+n]))
            if v[3][i+1]==v[2][i]+distance(bit_seg,table[6][4:4+n]):
                s[3][i+1]="c"
            else:
                s[3][i+1]="d"
    return s,v
def trace(data):
    s=data[0]
    v=data[1]
    num=len(s[0])
    i=0
    res=[]
    #print(v)
    value=10000000000000
    for j in range(1,len(v)):
        value=min(v[j][num-1],v[j-1][num-1],value)
        if value==v[j][num-1]:
            i=j
        elif value==v[j-1][num-1]:
            i=j-1
        else:
            i=i
    #print(i) 
    while num>1:
        state=s[i][num-1]
        if i==0 or i==1:
            res.append(0)
        else:
            res.append(1)
        if state=="a":
            i=0
        elif state=="b":
            i=1
        elif state=="c":
            i=2
        else:
            i=3
       # print(state)
       # print(i)
        num-=1
    res.reverse()
    return res
def conv_enc(bit,impulse):
    table=create_table(impulse)
    n=len(impulse)
    res=[]
    state="a"
    for i in range(len(bit)):
        if bit[i]==0:
            if state=="a":
                res+=(table[0][4:4+n])
                state=table[0][4+n]
            elif state=="b":
                res+=(table[1][4:4+n])
                state=table[1][4+n]
            elif state=="c":
                res+=(table[2][4:4+n])
                state=table[2][4+n]
            else:
                res+=(table[3][4:4+n])
                state=table[3][4+n]
        else:
            if state=="a":
                res+=(table[4][4:4+n])
                state=table[4][4+n]
            elif state=="b":
                res+=(table[5][4:4+n])
                state=table[5][4+n]
            elif state=="c":
                res+=(table[6][4:4+n])
                state=table[6][4+n]
            else:
                res+=(table[7][4:4+n])
                state=table[7][4+n]
    return res 
def conv_dec(bit,impluse):
    signal=decode(bit,impluse)
    return trace(signal)
def random_seq(n):
    res=[]
    for i in range(n):
        x=random.random()
        if x<0.5:
            res.append(0)
        else:
            res.append(1)
    return res
def random_filp(bit,p):
    for i in range(len(bit)):
        x=random.random()
        if x<p:
            if bit[i]==0:
                bit[i]=1
            elif bit[i]==1:
                bit[i]=0
    return bit     
def error_rate(l1,l2):
    count=0
    for i in range(len(l1)):
        if l1[i]!=l2[i]:
            count+=1
    return float(count)/len(l1)
            

In [5]:
g=[[1,0,0],[1,0,1],[1,1,1]]
bit=random_seq(100000)
bit_enc=conv_enc(bit,g)
bit_filp=random_filp(bit_enc,p=1)
bit_dec=conv_dec(bit_filp,g)
error_rate(bit,bit_dec)

0.5014

In [6]:
g=[[1,1,0],[1,0,1]]
bit=random_seq(100000)
bit_enc=conv_enc(bit,g)
bit_filp=random_filp(bit_enc,p=1)
bit_dec=conv_dec(bit_filp,g)
error_rate(bit,bit_dec)

0.5